# Introduction

Analysis of a set of simulation outputs that are the result of a parameter sweep.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os.path
import cPickle as pickle

In [ ]:
# Width of the page for calibrating fig_size.
# Approx. 16 for matplotlib backend %inline  8 for %notebook
page_width = 14
ax_height = 3

# Style of figures (default colors etc.): see https://matplotlib.org/gallery/style_sheets/style_sheets_reference.html
plt.style.use('default')

# Load Data

In [ ]:
# Paste output directories below (e.g use ctrl+c in Nautilus file manager)
outputs_clipboard = """
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780629.sonic-head_DA-depleted-v3_CTX-favg14_fburst5
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780630.sonic-head_DA-depleted-v3_CTX-favg14_fburst7
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780631.sonic-head_DA-depleted-v3_CTX-favg14_fburst9
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780632.sonic-head_DA-depleted-v3_CTX-favg14_fburst11
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780634.sonic-head_DA-depleted-v3_CTX-favg14_fburst15
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780635.sonic-head_DA-depleted-v3_CTX-favg14_fburst17
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780636.sonic-head_DA-depleted-v3_CTX-favg14_fburst19
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780637.sonic-head_DA-depleted-v3_CTX-favg14_fburst21
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780638.sonic-head_DA-depleted-v3_CTX-favg14_fburst23
/run/media/luye/Windows7_OS/Users/lkoelman/simdata-win/syn-v2_q1_const-rate_vary-freq/2018.08.02_job-780639.sonic-head_DA-depleted-v3_CTX-favg14_fburst25
"""
output_dirs = outputs_clipboard.strip().split('\n')

In [ ]:
# Analysis results for each simulation were saved as a dict in pickle format.
# Load them and make them accessible using the value of the sweep variable as key.
analysis_results= {}
sweep_var_name = None
for odir in output_dirs:
    exported_results = os.path.join(odir, 'analysis_results.pkl')
    with open(exported_results, 'rb') as f:
        results_dict = pickle.load(f)
        if sweep_var_name is None:
            sweep_var_name = results_dict['sweep_var_name']
        elif results_dict['sweep_var_name'] != sweep_var_name:
            raise ValueError(
                "Simulation results in {} do not have the intended sweep variable!"
                "\nMaybe one of the output directories you copied does not match the series.".format(odir))
        # Get sweep value
        import re
        match = re.search(r'[0-9]+$', odir)
        sweep_value = float(match.group(0))
        analysis_results[sweep_value] = results_dict

In [ ]:
print(analysis_results.keys())
print(analysis_results.values()[0].keys())

# Firing Rates

In [ ]:
# Plot mean firing rates
sweep_vals = np.array(sorted(analysis_results.keys()))
rates = [analysis_results[val]['mean_rate']['CTX'] for val in sweep_vals]

fig, ax = plt.subplots(figsize=(0.5*page_width, ax_height))

ax.plot(sweep_vals, rates, color='g', alpha=0.4)
# ax.plot(sweep_vals, rates, color='r', marker='+')
ax.bar(sweep_vals, rates, width=0.35, alpha=0.4, color='g')

ax.set_xlabel(sweep_var_name)
ax.set_ylabel('Mean firing rate (Hz)')
ax.set_title('Mean CTX firing rate for parameter sweep "{}"'.format(sweep_var_name))

ax.set_xticks(sweep_vals)
# ax.set_xticklabels(sweep_vals)

ax.set_yticks(np.arange(0, int(max(rates)+2), 5), minor=False)
ax.set_yticks(np.arange(0, int(max(rates)+2), 1.0), minor=True)
ax.grid(True, axis='y', which='major')

# PSD

In [ ]:
# Plot the PSD
sig_label = 'STN_Vm'
cmap = plt.get_cmap('cool')
line_colors = [cmap(x) for x in np.linspace(0, 1, len(analysis_results))] # https://stackoverflow.com/questions/8931268/using-colormaps-to-set-color-of-line-in-matplotlib

def compare_psd(sig_label):
    """ Plot PSD for given signal for each simulation / sweep variable. """
    fig, ax = plt.subplots(figsize=(0.75*page_width, ax_height))
    for i, (sweep_value, results) in enumerate(analysis_results.items()):
        freqs, psd = results['PSD'][sig_label]
        ax.plot(freqs, psd, color=line_colors[i], label='{} = {}'.format(sweep_var_name, sweep_value))

    ax.set_ylabel('Power ({})'.format(psd.units))
    ax.set_xlabel('frequency ({})'.format(freqs.units))
    ax.set_xlim((0, 50))
    ax.grid(True)
    # ax.set_yscale('log')
    ax.set_title('Welch PSD for {}'.format(sig_label))
    ax.legend(loc='upper right')

## PSD Shared Axis

In [ ]:
compare_psd('STN_Vm')
compare_psd('GPE_Vm')

## PSD Heatmap

In [ ]:
sig_label = "STN_Vm"

# Concatenate PSDs of different sweep values into matrix
freqs, psd = analysis_results.values()[0]['PSD'][sig_label]
freq_res = freqs[1] - freqs[0]
fmax = 100.0
nfreq = int(fmax/freq_res) + 1

sweep_vals = np.array(sorted(analysis_results.keys()))
d_sweep = min(np.diff(sweep_vals)) # resolution of sweep variable
sweep_axis = np.arange(sweep_vals.min(), sweep_vals.max()+d_sweep, d_sweep)
freq_axis = freqs[:nfreq]
sweep_psds = np.empty((nfreq, sweep_axis.size), dtype=float)

# Fill matrix
valid_columns = []
for sweep_value, results in analysis_results.iteritems():
    # col_id = int((sweep_value - sweep_axis.min())/d_sweep)
    col_id = np.where(sweep_axis == sweep_value)[0][0]
    valid_columns.append(col_id)
    freqs, psd = results['PSD'][sig_label]
    sweep_psds[:, col_id] = psd.magnitude[:nfreq]

# Fill invalid columns with NaN
invalid_columns = [i for i in range(sweep_axis.size) if i not in valid_columns]
sweep_psds[:, invalid_columns] = np.nan
psd_sweep = np.ma.masked_array(sweep_psds, np.isnan(sweep_psds)) # mark invalid/missing data

In [ ]:
# Draw heat map
cmap = plt.get_cmap('viridis') # e.g. jet, plasma: https://matplotlib.org/examples/color/colormaps_reference.html
cmap.set_bad(color='k') # color for masked values

fig, ax = plt.subplots(figsize=(0.5*page_width, ax_height))
sweep_plotaxis = np.concatenate((sweep_axis, [sweep_axis.max()+d_sweep])) # fix bug/feature in pcolormesh
plt.pcolormesh(sweep_plotaxis, freq_axis, psd_sweep, cmap=cmap)

# f_max = 50
# plt.ylim((0, f_max))
plt.colorbar()
# plt.clim(0, 20)
# cmap.set_bad('grey')
ax.set_xticks(sweep_axis + d_sweep/2.0) # otherwise they appear left of column
ax.set_xticklabels(sweep_axis)
ax.set_xlim((sweep_axis.min(), sweep_axis.max()+d_sweep))
ax.set_yticks(np.arange(0,50,1), minor=True)
ax.set_yticks(np.arange(0,55,5), minor=False)
ax.set_ylim((0, 50))
# TODO: find out why one column doesn't show

plt.ylabel('frequency (Hz)')
plt.xlabel(sweep_var_name)
plt.suptitle('Evolution of PSD for parameter sweep "{}"'.format(sweep_var_name))

## Bursting Power

In [ ]:
# Plot max power of bursting (look up PSD values at bursting frequency)

# Spectrogram

## Peak Power

Note that this might give a different picture than comparing simulations with PSD alone: a simulation with periodic synchronization may have higher transient peaks, whereas the PSD (average over time) is lower.

# Synchronization